<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2023/blob/main/Unidad_1/02_Recursion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

En esta sesión veremos algunos ejemplos clásicos donde las recursiones juegan un papel muy importante.

# Sucesión de Fibonacci

Es de las mas famosas dentro de las sucesiones definidas recursivamente. Los términos de la sucesión se definen por:

\begin{align}
F_0 = 0,\ F_1 = 1\\
F_n = F_{n-1} + F_{n-2}
\end{align}

Veamos algunos ejemplos relacionados con esta sucesión.

## Ejemplo 1

**Muestra que cualesquiera dos números consecutivos en la sucesión de Fibonacci son primos relativos.**

---

Probaremos por inducción que para todo entero positivo $n$, se cumple:

$$(F_n, F_{n-1}) = 1$$

donde $(x,y)$ es el máximo común divisor de $x$ y $y$.

**Demostración:**

* Caso base: Claramente:

$$(0, 1) = 1$$

* Hipótesis de inducción: 

$$(F_k, F_{k-1}) = 1$$

* Paso inductivo: Usando que $(a,b) = (a-b, b)$ Notemos que 

\begin{align}
(F_{k+1}, F_k) &= (F_{k+1} - F_k, F_k) \\
&= (F_{k-1}, F_k) \\
&= 1
\end{align}
Lo último por nuestra hipótesis de inducción. 

Concluimos que en efecto, $(F_n, F_{n-1}) = 1$ para todo entero positivo $n$.

## Ejemplo 2 

**Encuentra el residuo de $F_{100005}$ al ser dividido entre $23$.**

---

Como cada término depende únicamente de los dos anteriores, al considerar la sucesión dada por los residuos de la de Fibonacci, hay $23 \cdot 23$ parejas posibles de residuos módulo $23$. Por lo tanto, la sucesión de los residuos se va a ciclar a partir de cierto momento (queda como ejercicio probar que esta sucesión se cicla desde el principio). 

Encontrar la longitud del periodo de esta secuencia nos será útil para resolver este problema, pues si el periodo es $p$ únicamente restaría encontrar el residuo de $100005$ módulo $p$.

Encontremos entonces el periodo de la sucesión de los residuos módulo $23$.

In [30]:
res = [0, 1]
cur = (1, 1)

while cur != (1,0):
    res.append(cur[1])
    cur = (cur[1], (cur[0] + cur[1])%23)

len(res)

48

Ahora solo resta encontrar el residuo de $100005$ módulo $48$, y tomar el respectivo residuo de la sucesión. 

In [31]:
print("El residuo del 100005-ésimo número de Fibonacci módulo 23 es:", res[100005 % 48])

El residuo del 100005-ésimo número de Fibonacci módulo 23 es: 21


# Torres de Hanoi

Consideremos el problema de las torres de Hanoi con $n$ discos:

Se tienen $n$ discos de distintos tamaños y tres postes verticales donde se pueden poner estos discos. Inicialmente, se tienen todos los discos en el primer poste ordenados del más chico al más grande, con el más grande en la base. En cada paso se puede mover un disco que se encuentre hasta arriba de algún poste a cualquier otro poste siempre y cuando se coloque en un poste vacío o sobre un disco de mayor tamaño. Determina la menor cantidad de pasos necesarios para mover todos los discos del primer poste al tercero.

Veamos qué pasa cuando $n$ es chico. 

Se puede jugar en https://www.mathsisfun.com/games/towerofhanoi.html

* Si $n = 1$ es evidente que en un movimiento podemos lograr lo deseado.  

* Para $n = 2$, para poder poner el disco más grande en el tercer poste necesitamos que el disco más chico esté en el segundo poste, es decir, necesitamos al menos tres movimientos, uno para mover el disco chico al segundo poste, otro para mover el grande al tercer poste, y finalmente otro para poner el disco pequeño arriba del grande. 

* Para $n = 3$, se sigue un razonamiento similar. Necesitamos que los dos discos más chicos estén en el segundo poste (por el caso anterior requerimos al menos $3$ movimientos para esto), para después mover el disco más grande al tercer poste (un movimiento), y luego poner los discos más chicos encima de éste (otros $3$ movimientos), lo que nos da un total de $7$ movimientos.

Analizando estos primeros casos, podemos llegar a la conjetura de que el mínimo número de pasos necesarios para lograr lo deseado son $2^n - 1$. Probemos esto por inducción.

* Caso base. Los mencionados anteriormente.

* Hipótesis de inducción. Para $k$ discos, el menor número de pasos necesarios para mover todos los discos al tercer poste son $2^k - 1$.

* Paso inductivo. Con $k+1$ discos. Para poder mover el disco más grande al tercer poste, es necesario que todos los demás estén en el segundo poste, lo que por hipótesis de inducción requiere al menos $2^k -1$ pasos. Posteriormente se requiere al menos un paso para mover el disco más grande al tercer poste, y para concluir, como todos los demás discos tuvieron que estar en el segundo poste, moverlos al tercero nos tomará otros $2^k - 1$ pasos. Sumando, tenemos que se necesitan al menos:

$$(2^k - 1) + 1 + (2^k - 1) = 2^{k+1} - 1$$ 

pasos para lograr pasar todos los discos al tercer poste. Notemos que este paso inductivo también nos dice cómo ir haciendo los pasos para mostrar que $2^{k+1} - 1$ es suficiente para poder llegar al estado deseado.

Concluimos que el menor número de pasos para poder pasar los $n$ discos del primer al tercer poste son $2^n - 1$.

---

Ahora, veamos cómo se puede hacer un código que resuelva este problema en la menor cantidad de pasos posible. Para esto, diseñamos el siguiente sistema para codificar el estado de los discos:

* Los postes están representados por una lista de listas $P$. $P_i$ corresponde al $i$-ésimo poste, de izquierda a derecha.

* Los $n$ discos se denotan por números desde $1$ hasta $n$, en orden ascendente en relación a su tamaño ($1$ es el más chico, $n$ el más grande).

* $P_i$ contiene los números de los discos que están en el correspondiente poste, de abajo hacia arriba. Por ejemplo, si el segundo poste tiene los discos 1, 3, y 5, $P_2 = [5,3,1]$.

In [95]:
P = [[4, 3, 2, 1], [], []]
steps = 0

def pprint(P):
    n = sum([len(l) for l in P])
    s = '|'
    for l in P:
        for x in sorted(l)[::-1]:
            s += f' {x}'
        s += (n - len(l) + 1)*'  ' + '|'
    print(s)

    
def hanoi_solve(y, a, b):
    global steps
    if y == 1:
        P[a].pop()
        P[b].append(1)
        pprint(P)
    else:
        hanoi_solve(y-1, a, 3-a-b)
        P[a].pop()
        P[b].append(y)
        steps += 1
        pprint(P)
        hanoi_solve(y-1, 3-a-b, b)
    return

hanoi_solve(4, 0, 2)

| 4 3 2    | 1        |          |
| 4 3      | 1        | 2        |
| 4 3      |          | 2 1      |
| 4        | 3        | 2 1      |
| 4 1      | 3        | 2        |
| 4 1      | 3 2      |          |
| 4        | 3 2 1    |          |
|          | 3 2 1    | 4        |
|          | 3 2      | 4 1      |
| 2        | 3        | 4 1      |
| 2 1      | 3        | 4        |
| 2 1      |          | 4 3      |
| 2        | 1        | 4 3      |
|          | 1        | 4 3 2    |
|          |          | 4 3 2 1  |


## Ejercicio 1 

La función de Ackermann $A(m, n)$ está definida para $m, n$ positivos, como:

$$
\begin{align}
A(0, n) &= n+1\\
A(m+1, 0) &= A(m, 1)\\
A(m+1, n+1) &= A(m, A(m+1, n))
\end{align}
$$

Escribe una función que te permita calcularla para valores arbitrarios de $m, n$, y que regrese un error en caso de que $m$ o $n$ se salga del dominio.

*¡Cuidado! Esta función crece extremadamente rápido. Si quieres probarla, ejecútala para valores muy pequeños de $m$ y $n$.*

## Ejercicio 2

Muestra que al considerar la sucesión de Fibonacci módulo $n$, la sucesión obtenida se cicla desde el principio.

## Ejercicio 3

(Reto, no obligatorio). Muestra que los ciclos de menor longitud en la sucesión de Fibonacci módulo $n$ tienen $1, 2$ o $4$ ceros.